<a href="https://colab.research.google.com/github/Fitzpatrique/Deep-Learning/blob/main/Hands_on_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Artificial Neural Network

import the necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

In [ ]:
tf.__version__

'2.5.0'

Load the dataset into a pandas dataframe

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Machine Learning A-Z (Codes and Datasets)/Part 8 - Deep Learning/Section 39 - Artificial Neural Networks (ANN)/Python/Churn_Modelling.csv")

Perform EDA on the pandas dataframe

In [ ]:
print(df.head())

   RowNumber  CustomerId   Surname  ...  IsActiveMember EstimatedSalary Exited
0          1    15634602  Hargrave  ...               1       101348.88      1
1          2    15647311      Hill  ...               1       112542.58      0
2          3    15619304      Onio  ...               0       113931.57      1
3          4    15701354      Boni  ...               0        93826.63      0
4          5    15737888  Mitchell  ...               1        79084.10      0

[5 rows x 14 columns]


In [ ]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


#Data Preprocessing

In [ ]:
df.drop(labels=['RowNumber','CustomerId','Surname'], axis=1, inplace=True)

In [ ]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Perform Label Encoding

In [ ]:
geo_list = list(df['Geography'].unique())
gen_list = list(df['Gender'].unique())
print(geo_list,gen_list)

['France', 'Spain', 'Germany'] ['Female', 'Male']


In [ ]:
def france_label(geo):
  if geo == 'France':
    return 1
  else:
    return 0

def spain_label(geo):
  if geo == 'Spain':
    return 1
  else:
      return 0

def germany_label(geo):
  if geo == 'Germany':
    return 1
  else:
    return 0

def gen_label(gen):
  for i in range(len(gen_list)):
    if gen_list[i] == gen:
      return i

In [ ]:
df['France'] = df['Geography'].apply(france_label)
df['Spain'] = df['Geography'].apply(spain_label)
df['Germany'] = df['Geography'].apply(germany_label)
df['Gender'] = df['Gender'].apply(gen_label)

In [ ]:
df.drop(labels='Geography', axis=1, inplace=True)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Spain,Germany
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [ ]:
 cols = df.columns.tolist()
 print(cols)

['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited', 'France', 'Spain', 'Germany']


In [ ]:
cols = ['Gender', 'France', 'Spain', 'Germany','HasCrCard', 'IsActiveMember','CreditScore','Age', 'Tenure', 'Balance', 'NumOfProducts','EstimatedSalary', 'Exited']

In [ ]:
df = df[cols]

In [ ]:
df.head()

,Gender,France,Spain,Germany,HasCrCard,IsActiveMember,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Exited
0,0,1,0,0,1,1,619,42,2,0.00,1,101348.88,1
1,0,0,1,0,0,1,608,41,1,83807.86,1,112542.58,0
2,0,1,0,0,1,0,502,42,8,159660.80,3,113931.57,1
3,0,1,0,0,0,0,699,39,1,0.00,2,93826.63,0
4,0,0,1,0,1,1,850,43,2,125510.82,1,79084.10,0


Splitting the data set into the training set and testing set with scikit learn

In [ ]:
X = df.values[:,:-1]
y = df.values[:, -1]

print(X)
print(y)

[[0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 ... 8.3807860e+04
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 1.5966080e+05
  3.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 7.5075310e+04
  2.0000000e+00 9.2888520e+04]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 1.3014279e+05
  1.0000000e+00 3.8190780e+04]]
[1. 0. 1. ... 1. 1. 0.]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,6:] = sc.fit_transform(X_train[:,6:])
X_test[:,6:] = sc.transform(X_test[:,6:])

In [ ]:
print(X_train)

[[ 1.          0.          0.         ...  0.90982532 -0.91872163
  -0.56087335]
 [ 0.          0.          1.         ...  0.64950905 -0.91872163
  -0.156221  ]
 [ 1.          0.          1.         ... -1.22091009  0.79931788
   1.56921791]
 ...
 [ 0.          0.          0.         ...  0.69034266  0.79931788
  -0.14906952]
 [ 0.          1.          0.         ...  0.56499322 -0.91872163
  -0.7496959 ]
 [ 1.          0.          0.         ...  0.42602853 -0.91872163
  -1.71929584]]


#Building the ANN

Initialize the ANN

In [ ]:
ann = tf.keras.models.Sequential()

Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation= 'relu'))

Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation= 'relu'))

Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation= 'sigmoid'))

#Training the ANN

Compiling the ANN

In [ ]:
ann.compile(optimizer= 'adam',loss= 'binary_crossentropy' ,metrics= ['accuracy'])

Training the ANN on the dataset

In [ ]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5856 - accuracy: 0.7431
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4696 - accuracy: 0.8018
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4385 - accuracy: 0.8101
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4252 - accuracy: 0.8098
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4150 - accuracy: 0.8150
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4008 - accuracy: 0.8231
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3834 - accuracy: 0.8345
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3670 - accuracy: 0.8436
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3557 - accuracy: 0.8495
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3488 - accura

In [ ]:
print(ann.predict(X_test))
y_predict = ann.predict(X_test)

[[0.59071475]
 [0.72505105]
 [0.59895635]
 ...
 [0.14211884]
 [0.25376147]
 [0.9262406 ]]


In [ ]:
print(y_test)

[0. 0. 1. ... 1. 0. 1.]


#Making the predictions and evaluating the model

Predicting a single observation

In [ ]:
list1 = [[1,0,0,0,1,1]]
list1 = np.array(list1)
print(list1)

[[1 0 0 0 1 1]]


In [ ]:
list2 = sc.transform([[600,40,3,60000,2,50000]])
list2 = np.array(list2)
print(list2)

[[-0.52493139  0.09471269 -0.69884427 -0.26134615  0.79931788 -0.87196872]]


In [ ]:
list3 = np.concatenate((list1,list2), axis=1)
print(list3)

[[ 1.          0.          0.          0.          1.          1.
  -0.52493139  0.09471269 -0.69884427 -0.26134615  0.79931788 -0.87196872]]


In [ ]:
print(ann.predict(list3) > 0.5)

[[False]]


In [44]:
y_predict = (y_predict > 0.5)
print(np.concatenate((y_predict.reshape(len(y_predict), 1), y_test.reshape(len(y_test), 1)), 1))

[[1. 0.]
 [1. 0.]
 [1. 1.]
 ...
 [0. 1.]
 [0. 0.]
 [1. 1.]]


Making the confusion matrix

In [45]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_predict)
print(cm)
accuracy_score(y_test, y_predict)

[[1507   71]
 [ 219  203]]


0.855